# Generate QA Pairs based on the first 200 docs in Solution Ops


In [ ]:
import pandas as pd
import json
import tqdm
import glob
import os

%reload_ext dotenv
%dotenv

aoi_deployment_name = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
aoi_api_key = os.environ["aoi_api_key"]
azure_aoai_endpoint = os.environ["azure_aoai_endpoint"]

In [7]:
def create_qa_generation_prompt_from_chunk_context(chunk_text: str) -> list[dict]:
    system_prompt = """you are a prompt creator and have ability to generate new JSON prompts based on the given CONTEXT.
Generate 1 most relevant new prompt in valid json format according to "RESPONSE SCHEMA EXAMPLE" completely from scratch.
"RESPONSE SCHEMA EXAMPLE":
[
    {
        "role: "user",
        "content": "This is the generated prompt text",
    },
    {
        "role: "assistant",
        "content": "the expected, rightful and correct answer for the question"
    },
]
"""
    user_prompt = """The response must be valid JSON array containing two objects. The first object must contain the keys "role" and "content". The second object must also contain the keys "role" and "content".
    The response must follow the "RESPONSE SCHEMA EXAMPLE".
    The most important thing is that the response must be valid JSON ARRAY. DO NOT include anything other than valid schema.

    CONTEXT:

    """
    final_messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt + chunk_text + "\nEND OF CONTEXT"
        }
    ]
    return final_messages

In [25]:
def read_chunks():
    with open("./output/chunks-solution-ops-200.json", "r", encoding="utf-8") as file:
        chunks = json.load(file)
    print(chunks)
    return chunks

In [11]:
import openai
def generate_qa_from_chunk_text(chunk_text:str) -> dict:
    final_prompt = create_qa_generation_prompt_from_chunk_context(chunk_text)
    return final_prompt


In [38]:
import openai

openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_base = os.getenv(
    "AZURE_OPENAI_ENDPOINT"
)  # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = "azure"
openai.api_version = "2023-07-01-preview"  # this might change in the future


def call_aoai_gpt4(messages: list[dict]):
    response = openai.ChatCompletion.create(
        engine=aoi_deployment_name, messages=messages  # engine = "deployment_name".
    )

    return response.choices[0].message.content

In [47]:
#OUTRAGEOUSLY LONG: make it async later
def generate_qa_pairs() -> list:
    chunks = read_chunks()

    column_names = ["user_prompt", "output_prompt", "context"]
    final_df = pd.DataFrame(columns=column_names)

    i = 0
    for chunk_dict in chunks: 
        # chunk_id, chunk_dict in tqdm.tqdm(chunks_dict.items()):
        chunk_id = chunk_dict["chunkId"]
        if i <10:
            i += 1
            chunk_text = chunk_dict["chunkContent"]
            prompt_from_chunk_context = generate_qa_from_chunk_text(chunk_text)
            response = call_aoai_gpt4(prompt_from_chunk_context)
            
            try:
                response_dict = json.loads(response)
                for item in response_dict:
                    if item["role"] == "user":
                        user_prompt = item["content"]
                    if item["role"] == "assistant":
                        output_prompt = item["content"]

                data = {
                    "user_prompt": user_prompt,
                    "output_prompt": output_prompt,
                    "context": chunk_text,
                    "chunk_id": chunk_id,
                    "source": chunk_dict["source"]
                }

                final_df = final_df._append(data, ignore_index=True) #logic needs to be rewritten here to write better code to create DF (lambd/map)
            
            except Exception as e:
                print(f"Error: {e}")
        else:

            break
    final_df.to_json("./output/qa/solution-ops-200-qa.json", orient="records")
    return final_df

In [48]:
qa_paris = generate_qa_pairs()

[{'chunkId': 'chunk0_0', 'chunkContent': 'layout: forward\ntarget: https://aka.ms/solutionops\nrings:\n  - public\n\nforward', 'source': '..\\data\\docs\\index.md'}, {'chunkId': 'chunk1_0', 'chunkContent': "rings:\n  - public\n\nData Playbook\n\nThe Data Playbook provides enterprise software engineers with solutions, capabilities, and code developed to solve real-world problems. Everything in the playbook is developed with, and validated by, some of Microsoft's largest and most influential customers and partners.\n\n{% if extra.ring == 'internal' %}\nYou are invited to share your enterprise-grade production solutions as well. Refer to Contributing to the Solutions Playbook.\n\n{% endif %}\n\nData Solutions\n\nModern Data Warehouse solution\n{% if extra.ring == 'internal' %}\n\nData Mesh solution\n\nAnalytics and ML for enterprise business applications solution\n\nEnterprise Data Sharing solution\n{% endif %}\n\n{% if extra.ring == 'internal' %}\n\n{% else %}\n\n{% endif %}\n\nAbout the